# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download(['punkt','wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import re
import numpy as np
import pandas as pd 
import sys
import os
import re
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import classification_report

In [2]:
# load data from database
database_filepath = "DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
df = pd.read_sql_table("msgcategory", engine)


In [3]:

df.describe(include = 'all') 

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216,10170,26216,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
unique,NaN,26177,9630,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,#NAME?,Nap fe ou konnen ke apati de jodi a sevis SMS ...,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,20,13054,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,15224.82133,NaN,NaN,NaN,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,NaN,NaN,NaN,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,NaN,NaN,NaN,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
# List out the columns with % NAN values as these will be used for further analysis
   
print (round((df.isnull().sum() * 100/ len(df)),2).sort_values(ascending=False))

original                  61.21
direct_report              0.00
search_and_rescue          0.00
clothing                   0.00
shelter                    0.00
food                       0.00
water                      0.00
child_alone                0.00
military                   0.00
security                   0.00
medical_products           0.00
other_weather              0.00
medical_help               0.00
aid_related                0.00
offer                      0.00
request                    0.00
related                    0.00
genre                      0.00
message                    0.00
money                      0.00
missing_people             0.00
refugees                   0.00
aid_centers                0.00
cold                       0.00
earthquake                 0.00
fire                       0.00
storm                      0.00
floods                     0.00
weather_related            0.00
other_infrastructure       0.00
shops                      0.00
death   

In [5]:
# related column contains 3 values 0,1,2.With 1 containing majority of values.Leaving these values as is also 
#causes a value error in the modeling. As per recommendation from Udacity mentor in Knowledge forum, retaining only 2 values 
#in the related field as 2 contains minimal values
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [6]:
X = df['message'] #input variable
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url,url_place_holder_string)
        
    # Extract the word tokens from the provided text
    tokens = word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens
    

In [10]:


class StartingVerbExtractor (BaseEstimator, TransformerMixin):
    
    def starting_verb (self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB','VBP'] or first_word == 'RT':
                return True
        return False
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

pipeline1 = Pipeline([
('features', FeatureUnion([
('text_pipeline', Pipeline([
('vect', CountVectorizer(tokenizer=tokenize)),
('tfidf', TfidfTransformer())
])),
    

    
])),
('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

pipeline2 = Pipeline([
('features', FeatureUnion([
('text_pipeline', Pipeline([
('vect', CountVectorizer(tokenizer=tokenize)),
('tfidf', TfidfTransformer())
])),
    
('starting_verb'
, StartingVerbExtractor())
    
])),
('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
nltk.download('averaged_perceptron_tagger')
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fit = pipeline1.fit(X_train, y_train)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:

y_prediction_train = pipeline_fit.predict(X_train)
y_prediction_test = pipeline_fit.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      5002
               request       0.79      0.56      0.66      1132
                 offer       0.09      0.04      0.06        23
           aid_related       0.77      0.61      0.68      2750
          medical_help       0.64      0.25      0.36       503
      medical_products       0.73      0.34      0.46       326
     search_and_rescue       0.64      0.18      0.29       190
              security       0.17      0.03      0.05       114
              military       0.65      0.32      0.43       224
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.64      0.69       440
                  food       0.79      0.69      0.74       735
               shelter       0.78      0.53      0.63       582
              clothing       0.70      0.33      0.45        99
                 money       0.58      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.95      0.88     15092
               request       0.79      0.54      0.64      3342
                 offer       0.29      0.06      0.10        95
           aid_related       0.77      0.60      0.68      8110
          medical_help       0.65      0.28      0.39      1581
      medical_products       0.70      0.33      0.45       987
     search_and_rescue       0.68      0.23      0.34       534
              security       0.46      0.06      0.10       357
              military       0.65      0.35      0.46       636
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.64      0.70      1232
                  food       0.82      0.69      0.75      2188
               shelter       0.81      0.55      0.66      1732
              clothing       0.74      0.37      0.49       306
                 money       0.66    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
from sklearn.model_selection import GridSearchCV
parameters = {

'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
'classifier__estimator__n_estimators': [10, 20, 40]

                                 
 }    
cv = GridSearchCV(pipeline1, param_grid=parameters,scoring='f1_micro', n_jobs=-1)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.01, 0.02, 0.05], 'classifier__estimator__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
#Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

In [17]:
# Print classification report on test data
print('\n',classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.76      1.00      0.87      5002
               request       0.62      0.40      0.49      1132
                 offer       0.00      0.00      0.00        23
           aid_related       0.82      0.19      0.31      2750
          medical_help       0.70      0.11      0.19       503
      medical_products       0.74      0.12      0.21       326
     search_and_rescue       0.69      0.19      0.30       190
              security       0.00      0.00      0.00       114
              military       0.00      0.00      0.00       224
           child_alone       0.00      0.00      0.00         0
                 water       0.59      0.82      0.69       440
                  food       0.77      0.68      0.72       735
               shelter       0.87      0.28      0.42       582
              clothing       1.00      0.01      0.02        99
                 money       0.53    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [18]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.77      1.00      0.87     15092
               request       0.61      0.37      0.46      3342
                 offer       1.00      0.04      0.08        95
           aid_related       0.80      0.19      0.31      8110
          medical_help       0.62      0.12      0.20      1581
      medical_products       0.72      0.13      0.22       987
     search_and_rescue       0.62      0.19      0.29       534
              security       0.00      0.00      0.00       357
              military       0.00      0.00      0.00       636
           child_alone       0.00      0.00      0.00         0
                 water       0.57      0.85      0.68      1232
                  food       0.77      0.68      0.72      2188
               shelter       0.84      0.30      0.44      1732
              clothing       0.50      0.00      0.01       306
                 money       0.56    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
#Use pipeline2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fit = pipeline2.fit(X_train, y_train)

y_prediction_train = pipeline_fit.predict(X_train)
y_prediction_test = pipeline_fit.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5058
               request       0.78      0.56      0.65      1159
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.60      0.67      2726
          medical_help       0.61      0.26      0.37       538
      medical_products       0.65      0.34      0.45       329
     search_and_rescue       0.68      0.23      0.34       173
              security       0.18      0.03      0.05       105
              military       0.59      0.31      0.41       188
           child_alone       0.00      0.00      0.00         0
                 water       0.74      0.58      0.65       427
                  food       0.79      0.72      0.76       743
               shelter       0.80      0.57      0.67       591
              clothing       0.65      0.43      0.52        97
                 money       0.51      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15036
               request       0.79      0.56      0.65      3315
                 offer       0.38      0.06      0.11        82
           aid_related       0.77      0.60      0.68      8134
          medical_help       0.65      0.29      0.40      1546
      medical_products       0.69      0.34      0.46       984
     search_and_rescue       0.65      0.22      0.33       551
              security       0.59      0.09      0.16       366
              military       0.69      0.37      0.48       672
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.65      0.71      1245
                  food       0.82      0.70      0.76      2180
               shelter       0.79      0.56      0.65      1723
              clothing       0.72      0.44      0.54       308
                 money       0.64    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [21]:

with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.